## MODELADO CLASIFICACION NVICTIM_INTERV COMBINANDO OVERSAMPLING Y UNDERSAMPLING

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Carga de los datos ya codificados categóricos

In [2]:
df_premodelo = pd.read_csv('./madrid_accidentes_modelar_NVICTIM_INTERV_v1_0.csv',encoding='latin1')

df_premodelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86817 entries, 0 to 86816
Data columns (total 23 columns):
MAL_TIEMPO                               86817 non-null int64
MAL_ESTADO_SUELO                         86817 non-null int64
num_vehic_implic                         86817 non-null int64
conductor_menor                          86817 non-null int64
conductor_18_34                          86817 non-null int64
conductor_35_49                          86817 non-null int64
conductor_50_mas                         86817 non-null int64
RANGO_HORARIO_DE 00:00 A 7:00            86817 non-null int64
RANGO_HORARIO_DE 12:00 A 17:00           86817 non-null int64
RANGO_HORARIO_DE 17:00 A 21:00           86817 non-null int64
RANGO_HORARIO_DE 21:00 A 00:00           86817 non-null int64
RANGO_HORARIO_DE 7:00 A 12:00            86817 non-null int64
DIA_SEMANA_FIN DE SEMANA                 86817 non-null int64
DIA_SEMANA_LABORABLE                     86817 non-null int64
TIPO_ACCIDENTE_ATROPELL

## SELECCION CARACTERISTICAS

Probaremos ranking importancia características y eliminación recursiva.

In [3]:
X = df_premodelo.drop(columns='NVICTIM_INTERV')

y = df_premodelo['NVICTIM_INTERV']

X.shape

(86817, 22)

### Importancia característica, modelo RF

In [4]:
from sklearn.ensemble import RandomForestClassifier

model_RF_ranking = RandomForestClassifier(n_estimators=500,random_state=42)

model_RF_ranking.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [5]:
ranking_RF = pd.Series(model_RF_ranking.feature_importances_,index=X.columns)

ranking_RF.sort_values(ascending=False)

num_vehic_implic                         0.213344
TIPO_ACCIDENTE_COLISION MULTIPLE         0.119474
conductor_18_34                          0.087695
conductor_35_49                          0.081014
TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS    0.068596
conductor_50_mas                         0.064594
TIPO_ACCIDENTE_COLISION DOBLE            0.052645
RANGO_HORARIO_DE 00:00 A 7:00            0.044811
MAL_ESTADO_SUELO                         0.035470
MAL_TIEMPO                               0.034006
TIPO_ACCIDENTE_ATROPELLO                 0.025405
RANGO_HORARIO_DE 7:00 A 12:00            0.021158
DIA_SEMANA_FIN DE SEMANA                 0.019492
RANGO_HORARIO_DE 12:00 A 17:00           0.018086
DIA_SEMANA_LABORABLE                     0.018073
TIPO_ACCIDENTE_CAIDA BICICLETA           0.017980
RANGO_HORARIO_DE 17:00 A 21:00           0.017827
RANGO_HORARIO_DE 21:00 A 00:00           0.016384
TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO    0.016109
conductor_menor                          0.013364


### RFE  (ELIMINACION RECURSIVA)

In [6]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [7]:

model_RFE = RandomForestClassifier(random_state=42)
rfe = RFE(model_RFE, 18)

fit_RFE = rfe.fit(X, y)


In [8]:
ranking_RFE = pd.Series(X.columns.values, index=fit_RFE.ranking_)

ranking_RFE.sort_index()

1                               MAL_TIEMPO
1           TIPO_ACCIDENTE_CAIDA BICICLETA
1                 TIPO_ACCIDENTE_ATROPELLO
1                     DIA_SEMANA_LABORABLE
1                 DIA_SEMANA_FIN DE SEMANA
1            RANGO_HORARIO_DE 7:00 A 12:00
1         TIPO_ACCIDENTE_COLISION MULTIPLE
1           RANGO_HORARIO_DE 17:00 A 21:00
1           RANGO_HORARIO_DE 12:00 A 17:00
1            RANGO_HORARIO_DE 00:00 A 7:00
1                         conductor_50_mas
1                          conductor_35_49
1                          conductor_18_34
1            TIPO_ACCIDENTE_COLISION DOBLE
1                         num_vehic_implic
1                         MAL_ESTADO_SUELO
1    TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS
1    TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO
2           RANGO_HORARIO_DE 21:00 A 00:00
3                          conductor_menor
4              TIPO_ACCIDENTE_VUELCO_OTROS
5         TIPO_ACCIDENTE_CAIDA VIAJERO BUS
dtype: object

### Tabla resumen seleccion caracteristicas.

In [9]:
tabla_ranking = pd.DataFrame({'RF_importance': ranking_RF.sort_values(ascending=False).index, \
                             'orden_RFE': ranking_RFE.sort_index().index, 'atributo_RFE': ranking_RFE.sort_index().values})

tabla_ranking

,RF_importance,orden_RFE,atributo_RFE
0,num_vehic_implic,1,MAL_TIEMPO
1,TIPO_ACCIDENTE_COLISION MULTIPLE,1,TIPO_ACCIDENTE_CAIDA BICICLETA
2,conductor_18_34,1,TIPO_ACCIDENTE_ATROPELLO
3,conductor_35_49,1,DIA_SEMANA_LABORABLE
4,TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS,1,DIA_SEMANA_FIN DE SEMANA
5,conductor_50_mas,1,RANGO_HORARIO_DE 7:00 A 12:00
6,TIPO_ACCIDENTE_COLISION DOBLE,1,TIPO_ACCIDENTE_COLISION MULTIPLE
7,RANGO_HORARIO_DE 00:00 A 7:00,1,RANGO_HORARIO_DE 17:00 A 21:00
8,MAL_ESTADO_SUELO,1,RANGO_HORARIO_DE 12:00 A 17:00
9,MAL_TIEMPO,1,RANGO_HORARIO_DE 00:00 A 7:00


Nos quedamos con las de RFE

In [10]:
X.drop(columns=['RANGO_HORARIO_DE 21:00 A 00:00','conductor_menor', \
                'TIPO_ACCIDENTE_VUELCO_OTROS','TIPO_ACCIDENTE_CAIDA VIAJERO BUS'], inplace=True)



# 'RANGO_HORARIO_DE 7:00 A 12:00','RANGO_HORARIO_DE 12:00 A 17:00', 'RANGO_HORARIO_DE 17:00 A 21:00'\
#  'conductor_50_mas',
# ,'MAL_ESTADO_SUELO','MAL_TIEMPO', 'DIA_SEMANA_LABORABLE', \
# 'TIPO_ACCIDENTE_ATROPELLO','TIPO_ACCIDENTE_CAIDA BICICLETA','TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO', \
# 


X.shape

(86817, 18)

In [11]:
X.columns

Index(['MAL_TIEMPO', 'MAL_ESTADO_SUELO', 'num_vehic_implic', 'conductor_18_34',
       'conductor_35_49', 'conductor_50_mas', 'RANGO_HORARIO_DE 00:00 A 7:00',
       'RANGO_HORARIO_DE 12:00 A 17:00', 'RANGO_HORARIO_DE 17:00 A 21:00',
       'RANGO_HORARIO_DE 7:00 A 12:00', 'DIA_SEMANA_FIN DE SEMANA',
       'DIA_SEMANA_LABORABLE', 'TIPO_ACCIDENTE_ATROPELLO',
       'TIPO_ACCIDENTE_CAIDA BICICLETA',
       'TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS',
       'TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO',
       'TIPO_ACCIDENTE_COLISION DOBLE', 'TIPO_ACCIDENTE_COLISION MULTIPLE'],
      dtype='object')

### SEPARACIÓN CONJUNTOS TRAIN Y TEST ESTRATIFICADO

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

train_test_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in train_test_split.split(X, y):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    
    y_train, y_test = y[train_index], y[test_index]

In [13]:
y_train.value_counts(normalize=True)

LEVE           0.804561
MEDIA/GRAVE    0.195439
Name: NVICTIM_INTERV, dtype: float64

In [14]:
y_test.value_counts(normalize=True)

LEVE           0.804538
MEDIA/GRAVE    0.195462
Name: NVICTIM_INTERV, dtype: float64

## MODELOS DECISION TREES, RANDOM FOREST Y GRADIENT BOOST CLASSIFIER

### Comunes a todos los modelos

In [15]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score 

num_folds = 10
skf = StratifiedKFold(n_splits=num_folds)

scoring = 'roc_auc'
#'f1_macro' 'roc_auc'

## IMBALANCED LEARNING: COMBINAR 'SMOTE'  y 'NearMiss'

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

over = SMOTE(sampling_strategy='minority', random_state=42, n_jobs=-1)
num_trees = 300
modelo_DesTree = DecisionTreeClassifier(class_weight='balanced', random_state=42)
modelo_RF = RandomForestClassifier(n_estimators=num_trees, n_jobs=-1, random_state=42, class_weight='balanced')
modelo_GBC = GradientBoostingClassifier(n_estimators=num_trees, random_state=42)


ratio_under = list(np.arange(1,0.2,-0.1))

for ratio in ratio_under:
    
    print('RATIO:', ratio, '\n')
    
    under = NearMiss(sampling_strategy=ratio, n_jobs=-1)
    
    steps_DT = [('under', under), ('over', over), ('model', modelo_DesTree)]
    pipl_DT = Pipeline(steps=steps_DT)
    
    steps_RF = [('under', under), ('over', over), ('model', modelo_RF)]
    pipl_RF = Pipeline(steps=steps_RF)
    
    steps_GBC = [('under', under), ('over', over), ('model', modelo_GBC)]
    pipl_GBC = Pipeline(steps=steps_GBC)
    
    resultadosDesTree = cross_val_score(pipl_DT, X_train, y_train, cv=skf, scoring=scoring)
    print('DECISION TREE:','Media:',resultadosDesTree.mean(),' Desviación:',resultadosDesTree.std(),'\n')
    #print('Media:',resultadosDesTree.mean(),' Desviación:',resultadosDesTree.std())
    
    resultadosRF = cross_val_score(pipl_RF, X_train, y_train, cv=skf, scoring=scoring)
    print('RANDOM FOREST:','Media:',resultadosRF.mean(),' Desviación:',resultadosRF.std(),'\n')
    #print('Media:',resultadosRF.mean(),' Desviación:',resultadosRF.std())
    
    resultadosGBC = cross_val_score(pipl_GBC, X_train, y_train, cv=skf, scoring=scoring)
    print('GRADIENT BOOST CLASSIFIER:','Media:',resultadosGBC.mean(),' Desviación:',resultadosGBC.std(),'\n')
    #print('Media:',resultadosGBC.mean(),' Desviación:',resultadosGBC.std())
    
    print('-------------'*6,'\n')
    
    




RATIO: 1.0 

DECISION TREE: Media: 0.5898068780923829  Desviación: 0.006784020334921396 

RANDOM FOREST: Media: 0.5915952409673675  Desviación: 0.006800546255248061 

GRADIENT BOOST CLASSIFIER: Media: 0.6198404580209909  Desviación: 0.00623982539074168 

------------------------------------------------------------------------------ 

RATIO: 0.9 

DECISION TREE: Media: 0.5882287007666024  Desviación: 0.0073394487574090705 

RANDOM FOREST: Media: 0.5903964314469412  Desviación: 0.007484359208891113 

GRADIENT BOOST CLASSIFIER: Media: 0.6200256820632507  Desviación: 0.00648228921579059 

------------------------------------------------------------------------------ 

RATIO: 0.8 

DECISION TREE: Media: 0.5885312658488704  Desviación: 0.007273766901797345 

RANDOM FOREST: Media: 0.5903197022486133  Desviación: 0.007587787981950394 

GRADIENT BOOST CLASSIFIER: Media: 0.6202054645349898  Desviación: 0.005812110137292021 

-----------------------------------------------------------------------

**Los valores de AUC de todas las combinaciones alcanzan como máximo 0.62, lejos del por encima de 0.7 conseguido con OVERSAMPLING. Por lo tanto se desecha el uso de TECNICA COMBINADA OVER UNDER SAMPLING**